In [64]:
import polars as ps
import pymc3 as pm
import matplotlib.pyplot as plt
import seaborn as sb
from theano import shared, tensor
import arviz as az

In [65]:
type_map: list = [ps.Int32, ps.Datetime, ps.Int32, ps.Int32, ps.Int32, ps.Float32, ps.Float32, ps.Float32, ps.Float32,
                  ps.Float32, ps.Float32, ps.Float32, ps.Float32, ps.Float32, ps.Float32, ps.Float32, ps.Int32,
                  ps.Float32, ps.Float32, ps.Float32]

In [66]:
dataset: ps.DataFrame = ps.read_csv('full_dataset.csv', dtypes=type_map, encoding='utf8', n_threads=8, use_pyarrow=True)

In [67]:
dataset.head()

SiteId,PublishTime,AQI,Pollutant,Status,SO2,SO2_AVG,CO,CO_8hr,O3,O3_8hr,PM10,PM10_AVG,NO,NO2,NOx,WindDirec,WindSpeed,PM2.5,PM2.5_AVG
i32,datetime[μs],i32,i32,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i32,f32,f32,f32
67,2022-06-01 00:00:00,43,1,0,0.5,0.0,1.0,1.4,0.3,1.5,8.0,14.0,25.5,26.9,52.400002,188,1.7,5.0,8.0
43,2022-06-01 01:00:00,20,0,0,0.6,0.0,0.2,0.1,11.2,14.9,23.0,20.0,0.7,5.9,6.6,215,1.3,5.0,5.0
27,2022-06-01 03:00:00,18,0,0,0.9,1.0,0.1,0.1,12.4,16.799999,23.0,17.0,0.0,2.9,2.9,190,2.1,8.0,5.0
67,2022-06-01 05:00:00,19,1,0,0.4,0.0,0.4,0.8,4.9,1.9,18.0,15.0,13.0,10.2,23.299999,188,1.7,3.0,5.0
16,2022-06-01 06:00:00,26,1,0,0.6,0.0,0.5,1.0,2.9,1.9,21.0,18.0,5.5,9.6,15.1,168,1.2,8.0,8.0


# Features

In [68]:
features: list = ['AQI', 'Status', 'PM10', 'PM10_AVG', 'PM2.5_AVG']

In [69]:
variables: list = [shared(dataset[feature].to_numpy(writable=True)) for feature in features]

In [70]:
target = dataset['PM2.5'].to_numpy(writable=True)

# Model

In [71]:
with pm.Model() as predictor:
    intercept = pm.Normal(name='intercept', mu=0, sd=10)
    coefficients: list = [pm.Normal(name='coefficient{}'.format(i), mu=0, sd=10) for i in range(5)]
    error = pm.HalfNormal(name='error', sd=1)
    
    model = pm.Deterministic('model', (intercept + (coefficients[i] * variables[i] for i in range(5))))
    
    prediction = pm.Normal(name='prediction', mu=model, sd=error, observed=target)

    trace = pm.sample(tune=2000)
    ppc = pm.sample_posterior_predictive(trace, samples=2000)


You can find the C code in this temporary file: /var/folders/vv/93wphtyn4w15b8hssz_f1c500000gn/T/theano_compilation_error__2ijm2ht
library atest/envs/BDA/include/python3.9/Python.h:27:5: is not found.
library atest/envs/BDA/include/python3.9/Python.h:30: is not found.
library atest/envs/BDA/bin/../include/c++/v1/string.h:95:102: is not found.
library atest/envs/BDA/bin/../include/c++/v1/string.h:98:90: is not found.
library atest/envs/BDA/include/python3.9/Python.h:34: is not found.
library atest/envs/BDA/bin/../include/c++/v1/stdlib.h:150:34: is not found.
library atest/envs/BDA/bin/../include/c++/v1/stdlib.h:151:12: is not found.
library atest/envs/BDA/bin/../include/c++/v1/stdlib.h:154:34: is not found.
library atest/envs/BDA/bin/../include/c++/v1/stdlib.h:156:12: is not found.
library atest/envs/BDA/include/python3.9/Python.h:36:10: is not found.


Exception: ('Compilation failed (return status=1): In file included from /Users/hermeschen/.theano/compiledir_macOS-13.5-x86_64-i386-64bit-i386-3.9.16-64/tmpe5g7p5r4/mod.cpp:1:. /Users/hermeschen/.pyenv/versions/miniconda3-latest/envs/BDA/include/python3.9/Python.h:27:5: error: "Python.h requires that stdio.h define NULL.". #   error "Python.h requires that stdio.h define NULL.".     ^. In file included from /Users/hermeschen/.theano/compiledir_macOS-13.5-x86_64-i386-64bit-i386-3.9.16-64/tmpe5g7p5r4/mod.cpp:1:. In file included from /Users/hermeschen/.pyenv/versions/miniconda3-latest/envs/BDA/include/python3.9/Python.h:30:. /Users/hermeschen/.pyenv/versions/miniconda3-latest/envs/BDA/bin/../include/c++/v1/string.h:95:102: error: unknown type name \'size_t\'. inline _LIBCPP_HIDE_FROM_ABI _LIBCPP_PREFERRED_OVERLOAD const void* memchr(const void* __s, int __c, size_t __n) {.                                                                                                      ^. /Users/herm

In [ ]:
az.plot_trace(trace, var_names=['intercept', 'coefficient0', 'coefficient1', 'coefficient2', 'coefficient3', 'coefficient4', 'error'])